# Aufgabe5 – Interpretation von Modellen

## Notebook Beschreibung
...

## Bibliotheken importieren und Notebook setup

In [ ]:
# Modul Pfad setzen
import sys
sys.path.append('./module')


# Bibliotheken importieren
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import LinearSVC


## Analyse und Datenvorverarbeitung

Aufgabe 5a)  
Lesen Sie den Datensatz Hdma.csv ein und machen Sie sich mit den Daten vertraut.

In [ ]:
# Load Dataset from csv (use ; as separator)
data = pd.read_csv('./data/Hdma.csv', sep=';')
# Display first line to test if data was loaded correctly
data.head()

In [ ]:
# Show dataframe size
data.shape

Der Datensatz besteht aus 2381 Datenpunkten mit jeweils 13 Merkmalen

In [ ]:
data.isnull().any()

Die Spalten "pbcr" und "self" enthalten fehlende Daten.

In [ ]:
data[data.isnull().any(axis=1)]

Nur in einer Zeile ist "pbcr" und "self" gleich NaN. Da es nur eine einzige Zeile mit fehlenden Daten gibt, haben wir uns entschieden diese aus dem Datensatz zu entfernen.

In [ ]:
data = data.drop(2380)
data.shape

### Art der Merkmale 

In [ ]:
data.columns

- `dir` numerisch-kontinuierlich 
- `hir` numerisch-kontinuierlich
- `lvr` numerisch-kontinuierlich
- `ccs` kategorisch-ordinal
- `mcs` kategorisch-ordinal
- `pbcr` kategorisch-nominal
- `dmi` kategorisch-nominal
- `self` kategorisch-nominal
- `single` kategorisch-nominal
- `uria` numerisch-kontinuierlich
- `condo` kategorisch-nominal
- `black` kategorisch-nominal
- `deny` kategorisch-nominal


Die Zielvariable der Klassifikation ist 'deny' welche angibt ob der Kreditantrag abgelehnt wurde. Es soll bestimmt werden welche Merkmale besonderen Einfluss auf diese haben. Im Speziellen soll bestimmt werden, ob es eine rassistische Diskriminierung gibt. Dafür muss das Merkmal 'black' betrachtet werden

In [ ]:
sns.countplot(x='deny',data=data)
plt.show()

In [ ]:
# Ratio of denials in the dataset
data['deny'].value_counts()[1] / data.shape[0]

Erkenntnis: "Imbalanced Dataset"

In [ ]:
nominal_features = ['pbcr', 'dmi', 'self', 'single', 'condo', 'black', 'deny']
labelencoder = LabelEncoder()
data[nominal_features] = data[nominal_features].apply(labelencoder.fit_transform)
data.head()

In [ ]:
continuous_features = ['dir', 'hir', 'lvr']
for feature in continuous_features:
    plt.figure(figsize=(14, 4))
    sns.boxplot(data[feature])
    plt.show()

In [ ]:
outliers = data[data["dir"]>2]
outliers.head()

In [ ]:
data.drop(outliers.index, inplace=True)
data.shape

In [ ]:
correlation_matrix = data.corr()
fig = plt.figure(figsize=(12,9))
sns.heatmap(correlation_matrix,vmin=-1,vmax=1,annot=True)
plt.show()

In [ ]:
# Create numpy arrays for features and target
X = data.drop("deny", axis=1)
y = data["deny"]
feature_names = list(X.columns)

# Create 80/20 train val split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

# Print resulting dataset sizes
print("Shape X_train", X_train.shape)
print("Shape y_train", y_train.shape)
print("Shape X_test", X_test.shape)
print("Shape y_test", y_test.shape)

## Logistische Regression

Skalieren der Daten für die logistische Regression

In [ ]:
# Use StandardScaler to transform data to zero mean and unit variance
sc = StandardScaler()
# Fit and transform on training data
X_train_scaled = sc.fit_transform(X_train)
# Only apply transformation to test data
X_test_scaled = sc.transform(X_test)

Logistische Regression.

`class_weight = 'balanced'` sollte gesetzt werden, da es sich hierbei um einen unbalancierten Datensatz handelt.

In [ ]:
classifier_lregression = LogisticRegression(solver='lbfgs', class_weight = 'balanced')
classifier_lregression.fit(X_train_scaled, y_train)
print(f1_score(y_train, classifier_lregression.predict(X_train_scaled)))
print(f1_score(y_test, classifier_lregression.predict(X_test_scaled)))
print(f1_score(y_train, np.zeros_like(y_train)))
print(f1_score(y_test, np.zeros_like(y_test)))

print(balanced_accuracy_score(y_test, classifier_lregression.predict(X_test_scaled)))

In [ ]:
list(zip(data.columns.to_list(), classifier_lregression.coef_.squeeze().tolist()))

## Vergleich verschiedener Modelle

Instanziieren der genannten Klassifikatoren

In [ ]:
classifier_dtree = DecisionTreeClassifier(criterion = 'entropy')
classifier_rforest = RandomForestClassifier(n_estimators = 10, random_state=0)
classifier_adaboost = AdaBoostClassifier()
classifier_nb = ComplementNB() #particularly suited for imbalanced data sets
classifier_svm = LinearSVC(max_iter = 10000)

classifiers = [classifier_dtree, classifier_rforest, classifier_adaboost, classifier_nb, classifier_svm]

###### Trainieren der Klassifikatoren

In [ ]:
for classifier in classifiers:
    classifier.fit(X_train, y_train)

###### Vergleich der Feature Importances

In den folgenden Zellen wird ein Vergleich der Feature Importances der einzelnen Klassifikatoren durchgeführt.

Die Feature Importance, Koffizienten und _log_ der Wahrscheinlichkeit eines Features einer Klasse sind nicht direkt vergleichbar. Das liegt daran, dass sie unterschiedliche Aussagen treffen:

- Feature Importance
  - Attribut `feature_importances_` in `DecisionTreeClassifier`, `RandomForestClassifier` und `AdaBoostClassifier`)
  - Basiert darauf, wie sehr ein Attribut dazu beiträgt, den sog. "Information Gain" in einem Knoten eines Entscheidungsbaums zu erhöhen bzw. die Entropie der Daten in dessen Kind-Knoten zu verringern. Bei Random Forest wird der durchschnitt über alle Entscheidungsbäume für jedes Attribut berechnet. 


- Koffizienten
  - Attribut `coef_` in `LogisticRegression` und `LinearSVC`
  - Die Koeffizienten, die bei der Logistischen Regression bzw. beim Trainieren einer Linearen SVM gelernt für die einzelnen Merkmale gelernt wurden.


- _log_ der Wahrscheinlichkeit eines Features einer Klasse
  - Attribut `feature_log_prob_` in `ComplementNB`
  - 
  
Aufgrund dieser Informationen ist lediglich die Reihenfolge der Attribute nach ihrer Wichtigkeit im jeweiligen Klassifikator vergleichbar.

In [ ]:
# Define Method to sort the feature weightings and zip them with the feature names
def zip_and_sort(feature_weightings):
    for feature, weight in sorted(zip(feature_names, feature_weightings), key=lambda x: abs(x[1]), reverse = True):
        print('{:8s}{:.2f}'.format(feature, weight))

In [ ]:
# Print sorted feature weightings for every classifier
for feature_weightings in [classifier_lregression.coef_[0],
                          classifier_dtree.feature_importances_,
                          classifier_rforest.feature_importances_,
                          classifier_adaboost.feature_importances_,
                          classifier_nb.feature_log_prob_[1],
                          classifier_svm.coef_[0]]:
    zip_and_sort(feature_weightings)
    print()

###### Vorhersage im Test-Datensatz mithilfe der Klassifikatoren

In [ ]:
class_name = 'Most Common Class'
score = balanced_accuracy_score(y_test, np.zeros_like(y_test))
print('{:25s}{:.2f}\n'.format(class_name, score))

class_name = type(classifier_lregression).__name__
score = balanced_accuracy_score(y_test, classifier_lregression.predict(X_test_scaled))
print('{:25s}{:.2f}\n'.format(class_name, score))

for classifier in classifiers:
    class_name = type(classifier).__name__
    score = balanced_accuracy_score(y_test, classifier.predict(X_test))
    print('{:25s}{:.2f}'.format(class_name, score))
